In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import svd

In [5]:
from urllib.parse import urlparse, parse_qs

#if i want to include SMA/stock data for same company, need to scrape function not just symbol
def TimeSeriesDaily_df(stock_name):
    """
    Read a CSV file from a URL and return a DataFrame with symbol as the last column.
    """
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_name}&apikey=98WO4EK4CGOBTPQR&datatype=csv'
    qs = parse_qs(urlparse(url).query)
    symbol = qs["symbol"][0]
    df = pd.read_csv(url)
    df['symbol'] = symbol
    return df
    

In [ ]:
#Example: AAPL stock past 100 days daily time series data
aapl_df = TimeSeriesDaily_df('AAPL')
aapl_df.head()

,timestamp,open,high,low,close,volume,symbol
0,2025-05-12,210.97,211.2679,206.7500,210.79,63775814,AAPL
1,2025-05-09,199.00,200.5399,197.5350,198.53,36453923,AAPL
2,2025-05-08,197.72,200.0500,194.6796,197.49,50478872,AAPL
3,2025-05-07,199.17,199.4400,193.2500,196.25,68616943,AAPL
4,2025-05-06,198.21,200.6500,197.0200,198.51,51216482,AAPL


In [6]:
msft_df = TimeSeriesDaily_df('MSFT')
msft_df.head()

,timestamp,open,high,low,close,volume,symbol
0,2025-05-12,445.94,449.37,439.7800,449.26,22821935,MSFT
1,2025-05-09,440.00,440.74,435.8800,438.73,15324233,MSFT
2,2025-05-08,437.93,443.67,435.6600,438.17,23491330,MSFT
3,2025-05-07,433.84,438.12,431.1103,433.35,23307241,MSFT
4,2025-05-06,432.20,437.73,431.1700,433.31,15104204,MSFT


In [ ]:
#would need to choose which data I want to use (open, high, low, close volumn) and it's easy to make a function that makes a series of this data
#Example: Close price
aapl_close = aapl_df['close'].rename('AAPL close')
msft_close = msft_df['close'].rename('MSFT close')
date = aapl_df['timestamp']
data = [date, aapl_close, msft_close]

In [24]:
#take multiple data (dtype pd.Series) and make into a data matrix
def make_data_matrix(data: list[pd.Series]):
    """ 
    Make M1 and M2 data matrices from a list of Series. The first element of the list
    must be the dates (time stamp) and the rest are the data series.
    """
    M1_df = pd.DataFrame(data[1:])
    M1_df.index = [s.name for s in data[1:]]
    M1_df.columns = data[0]
    A = M1_df.to_numpy()
    M1 = A[:, :-1]
    M2 = A[:, 1:]
    return M1, M2


In [21]:
M1, M2 = make_data_matrix(data)


In [43]:
M1_centered = M1 - np.mean(M1)
U, s, Vt = np.linalg.svd(M1_centered)
S = np.diag(s)
s_inv = np.linalg.inv(S)
A_tild = U.T@M2@Vt.T@s_inv


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 99)